In [9]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
from functions.spotipyTools import *
from dotenv import load_dotenv
import os
import openpyxl

In [10]:
load_dotenv()

True

In [11]:
saved_tracks = pd.read_parquet('LatestPlayedTracks.parquet')

In [12]:
e_client_id = os.environ["client_id"]
e_client_key = os.environ["client_key"]
e_playlist_id = os.environ["playlist_id"]
e_user_id = os.environ["user_id"]

sp_login = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=e_client_id,
                                               client_secret=e_client_key,
                                               redirect_uri="http://localhost:8888/callback", # Need to whitelist the URL
                                               scope="user-read-recently-played"))

In [13]:
recently_played = sp_login.current_user_recently_played()

In [14]:
all_tracks, dates_played = get_recent_tracks(sp_login, 300)

In [15]:
track_info_list = []

for item in all_tracks:
  track_info_list.append(get_track_info(item, sp_login))

df_tracks = pd.DataFrame(track_info_list, columns=['name', 'artist_name', 'album_name', 'album_date', 'album_popularity', 'track_duration', 'danceability', 'energy', 'instrumentalness', 'liveness', 'tempo','artist_url'])

df_tracks['date_played'] = dates_played

In [17]:
df_tracks = pd.concat([df_tracks, saved_tracks]).drop_duplicates()

In [19]:
df_tracks

,name,artist_name,album_name,album_date,album_popularity,track_duration,danceability,energy,instrumentalness,liveness,tempo,artist_url,date_played
0,All My Own Stunts,Arctic Monkeys,Suck It and See,2011-06-06,48,232280,0.511,0.943,0.018100,0.1870,127.283,https://open.spotify.com/artist/7Ln80lUS6He07X...,2022-08-05T14:57:44.609Z
1,I Forgot to Take My Meds Today,Prince Daddy & the Hyena,I Thought You Didn't Even Like Leaving,2016-09-23,46,145215,0.403,0.938,0.000000,0.1930,109.934,https://open.spotify.com/artist/3EYgeLZ0vtVHKk...,2022-08-05T14:53:52.531Z
2,The Immateria,Burst,Origo,2005-02-07,8,322226,0.281,0.950,0.048300,0.1500,147.505,https://open.spotify.com/artist/7DQxsJbGUe0mwv...,2022-08-05T14:51:26.496Z
3,Good Advice,The Growlers,Chinese Fountain,2014-09-23,39,204906,0.447,0.822,0.005760,0.2730,137.023,https://open.spotify.com/artist/2RhgnQNC74QoBl...,2022-08-05T14:46:04.237Z
4,Everywhere,Kindling,Hush,2017-11-10,4,199297,0.370,0.850,0.763000,0.1020,104.330,https://open.spotify.com/artist/6fGIltjfZeNQ5j...,2022-08-05T14:42:38.553Z
5,Speak of the Devil,Black Pistol Fire,Deadbeat Graffiti,2017-09-29,45,299400,0.479,0.613,0.435000,0.0900,95.181,https://open.spotify.com/artist/0Nrwy16xCPXG8A...,2022-08-05T14:39:18.251Z
6,All the Same,Josh Homme,The Dangerous Lives of Altar Boys (Original Mo...,2002-01-20,36,285080,0.316,0.586,0.000125,0.0874,143.815,https://open.spotify.com/artist/03xb2BUdIFzuRQ...,2022-08-05T14:07:53.388Z
7,"Harder, Better, Faster, Stronger",Pomplamoose,"Harder, Better, Faster, Stronger",2019-10-16,42,248493,0.810,0.723,0.001100,0.0864,115.019,https://open.spotify.com/artist/5dHfLBNU8zoypg...,2022-08-05T14:03:07.432Z
8,Bridge Burning,Foo Fighters,Wasting Light,2011-04-12,56,286466,0.420,0.914,0.028200,0.2200,170.199,https://open.spotify.com/artist/7jy3rLJdDQY21O...,2022-08-05T13:56:52.423Z
9,In My Mind,The Amazons,The Amazons (Deluxe),2017-05-26,49,230333,0.320,0.892,0.000073,0.0899,77.508,https://open.spotify.com/artist/7243txmysJ4KbR...,2022-08-05T13:52:06.819Z


In [18]:
df_tracks.to_parquet('LatestPlayedTracks.parquet', engine='pyarrow')